In [1]:
pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.0 MB/s eta 0:00:00


In [2]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 50.6 MB/s eta 0:00:00


In [3]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
import mysql.connector
from mysql.connector import Error
from datetime import datetime

In [4]:
consumer = KafkaConsumer(
    'demo',
    bootstrap_servers=['13.71.31.242:9092'], #add your IP here
    auto_offset_reset='latest',  # Start from the earliest message
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [5]:
try:
  conn = mysql.connector.connect(
      host='13.71.31.242',    # Remote MySQL server address
      user='stock',           # MySQL username
      password='stock',       # MySQL password
      database='stockdb',       # The database you want to connect to
      port=3306               # Default MySQL port (you can change it if necessary)
  )
except Error as e:
  print(f"Error connecting to MySQL: {e}")

In [ ]:
while True:
  sleep(10)
  msg = consumer.poll(timeout_ms=10000)
  if msg:
    for topic_partition, messages_list in msg.items():
      print("\n\nBatch count: ", len(messages_list), "\n\n")

    # for message in messages_list:
    #   print(f"Consumed: {message.value}")

    cursor = conn.cursor()
    # Insert statement
    insert_query = """
    INSERT INTO StockData (IndexName, Date, Open, High, Low, Close, AdjClose, Volume, CloseUSD)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    for message in messages_list:
      print(f"Consumed: {message.value}")

      json_object = message.value

      json_object['Date'] = datetime.strptime(json_object['Date'], '%Y-%b-%d').strftime('%Y-%m-%d')

      cursor.execute(insert_query, (
          json_object['Index'],
          json_object['Date'],
          json_object['Open'],
          json_object['High'],
          json_object['Low'],
          json_object['Close'],
          json_object['Adj Close'],  # Use underscore for the column name
          json_object['Volume'],
          json_object['CloseUSD']
      ))

    conn.commit()
    cursor.close()
  else:
    print("No message from producer")




Batch count:  1 


Consumed: {'Index': 'NYA', 'Date': '2002-OCT-08', 'Open': 4591.330078, 'High': 4591.330078, 'Low': 4591.330078, 'Close': 4591.330078, 'Adj Close': 4591.330078, 'Volume': 1938430000.0, 'CloseUSD': 4591.330078}


Batch count:  10 


Consumed: {'Index': 'NYA', 'Date': '1980-JUN-27', 'Open': 701.460022, 'High': 701.460022, 'Low': 701.460022, 'Close': 701.460022, 'Adj Close': 701.460022, 'Volume': 0.0, 'CloseUSD': 701.460022}
Consumed: {'Index': 'NYA', 'Date': '2008-DEC-10', 'Open': 5522.470215, 'High': 5676.950195, 'Low': 5522.470215, 'Close': 5631.069824, 'Adj Close': 5631.069824, 'Volume': 5942130000.0, 'CloseUSD': 5631.069824}
Consumed: {'Index': 'NYA', 'Date': '1986-JAN-21', 'Open': 1257.640015, 'High': 1257.640015, 'Low': 1257.640015, 'Close': 1257.640015, 'Adj Close': 1257.640015, 'Volume': 0.0, 'CloseUSD': 1257.640015}
Consumed: {'Index': 'HSI', 'Date': '2002-JUL-17', 'Open': 10336.78027, 'High': 10362.45996, 'Low': 10213.01953, 'Close': 10335.12012, 'Adj Close'

In [ ]:
conn.close()